# Introduction

The VaR_Analyzer takes in Ex-Post Data (Damage Case Database) and Ex-Ante Data (Workshop Results).  
It aids in the statistical analysis of said data and arrives at a final result with a VaR simulation for operational risks.  
After running the Import block, each block in the file can be run separately.

Ex-Post Data resembles a Damage Case Database and should contain the amount of damage done, as well as a Time Stamp.

Ex_Ante Data resembles results worked out with insiders in the firm to asses potential future damages. For each future scenario, the data should contain the expected yearly frequency of the event, the typical damage caused by the event (most likely amount of damage to occur), and the extreme damage (potential damage relating to the 99%-Quantile).

# Data Import

Import own data or use/generate example data.

Ex-Post Data should be structured with the headings:  
'Event Name' ; 'Time Stamp' ; 'Loss'

Ex-Ante Data should be structured with the headings:  
'Scenario' ; 'Yearly Frequency' ; 'Typical Damage' ; 'Extreme Damage'

For further reference see the structure of ._example_data in \data.

In [ ]:
import src.exploratory_analysis
import src.distribution_estimation
import src.generate_test_data
import src.var_analysis
import src.jupyter_notebook_functions
from IPython.display import display, Markdown
import pandas as pd

# Import own data here or use example data
exPost_filePath = 'data/exPost_example_data.xlsx'
exAnte_filePath = 'data/exAnte_example_data.xlsx'

# Uncomment to generate new example data
#%run src\generate_test_data.py

exPost_df = pd.read_excel(exPost_filePath)
exPost_df['Time Stamp'] = pd.to_datetime(exPost_df['Time Stamp'])
exAnte_df = pd.read_excel(exAnte_filePath)

# Exploratory Analysis (Ex-Post)

This section provides an initial analyis of the Ex-Post Database, which can later aid in the estimation of the underlying distribution.

### Losses per Year (Discrete)

In [ ]:
src.exploratory_analysis.exploratory_statistics_discrete(exPost_df['Time Stamp'].dt.year.value_counts())

### Loss severity per Loss (Continuous)

In [ ]:
src.exploratory_analysis.exploratory_statistics_continuous(exPost_df['Loss'])

# Distribution Estimation

This section provides assistance in finding the best fitting underlying distribution for the Data. Both the discrete distribution of losses per year, and a continous distribution resembling the severity of each loss can be fitted.

For Ex-Post Data, the tool provides a dropdown menue, where several theoretical distributions can be fitted to each of the underlying distributions. It outputs the results of several statistical tests and displays a Quantile-to-Quantile plot, in order to visually assess the goodness of fit.
The best fitting parametrization is automatically selected and displayed for each theoretical distribution.

Since Ex-Ante Data is constructed from workshop results, each scenario is plotted with individual lognormal and poisson distributions and just the parameters are displayed.

## Ex-Post

### Losses per Year (Discrete)

In [ ]:
dropdown_discrete = src.jupyter_notebook_functions.create_discrete_dropdown()
display(dropdown_discrete)

dropdown_discrete.observe(lambda change: src.jupyter_notebook_functions.on_dropdown_discrete_change(change, exPost_df['Time Stamp'].dt.year.value_counts()), names='value')
src.jupyter_notebook_functions.on_dropdown_discrete_change({'type': 'change', 'name': 'value', 'new': dropdown_discrete.value}, exPost_df['Time Stamp'].dt.year.value_counts())

### Loss severity per Loss (Continuous)

In [ ]:
dropdown_continuous = src.jupyter_notebook_functions.create_continuous_dropdown()
display(dropdown_continuous)

dropdown_continuous.observe(lambda change: src.jupyter_notebook_functions.on_dropdown_continuous_change(change, exPost_df['Loss']), names='value')
src.jupyter_notebook_functions.on_dropdown_continuous_change({'type': 'change', 'name': 'value', 'new': dropdown_continuous.value}, exPost_df['Loss'])

## Ex-Ante

In [ ]:
src.distribution_estimation.exAnte_logNormalDistribution_parameters(exAnte_df)

# VaR - Simulation

This section simulates and displays yearly damages and calculates the VaR for both Ex-Post and Ex-Ante Data.

The Ex-Post simulation requires a theroetical discrete and continuous distribution as input. To find the correct distributions, utilize the sections above.  
For display reasons, the distribution names have to be provided manually.

The simulation results for both Ex-Post and Ex-Ante Data are saved to .xlsx and can be found in the directory data/.

## Ex-Post

Discrete Distributions: 'Poisson', 'Geometric', 'Negative Binomial'

Continuous Distributions: 'Lognormal', 'Normal', 'Exponential', 'Gamma', 'Loggamma', 'Beta', 'Logistic', 'Gumbel_r', 'Gumbel_l', 'Weibull', 'Pareto', 'Cauchy'

In [ ]:
# Number of simulation steps n | Distribution for Losses per Year (Discrete) | Distribution for Severity of Loss (Continuous)
n = 5000
discrete_dist = 'Poisson'
continuous_dist = 'Lognormal'

src.var_analysis.perform_exPost_simulations(exPost_df, discrete_dist, continuous_dist, n)

## Ex-Ante

In [ ]:
# Number of simulation steps
n = 5000

src.var_analysis.perform_exAnte_simulations(exAnte_df, n)

# VaR - Result

This section combines both simulation results and calculates the total VaR given the weight of the exAnte simulation relative to the exPost simulation.

CAREFUL: While this section can be run independently, it relies on the simulation results from the previous section, which are stored in data/.

In [ ]:
# Weight of Ex-Ante Estimation compared to Ex-Post
w_exAnte = 0.5

VaR = src.var_analysis.combine_simulation_results('data/exAnte_simulation_results.xlsx', 'data/exPost_simulation_results.xlsx', w_exAnte)
display(Markdown(f'## Total Value at Risk (99.9%): **{VaR}**'))